**實例：使用 Cirq 建立簡單的量子電路**

Cirq 是谷歌主導的開源量子計算函式庫，可以幫助我們方便的建立量子電路並模擬測量結果（我們在下一節介紹 TensorFlow Quantum 的時候還會用到它）。Cirq 是一個 Python 函式庫，可以使用 pip install cirq 進行安裝。以下程式碼實現了上一個章節所建立的兩個簡單的量子電路，並分別進行了 20 次的模擬測量。

In [6]:
!pip install cirq
!pip install -q tensorflow-quantum
!pip install --upgrade https://pypi.python.org/packages/b2/30/ab593c6ae73b45a5ef0b0af24908e8aec27f79efcda2e64a3df7af0b92a2/protobuf-3.1.0-py2.py3-none-any.whl#md5=f02742e46128f1e0655b44c33d8c9718

     |████████████████████████████████| 348kB 3.4MB/s 
ERROR: tensorflow 2.3.0 has requirement protobuf>=3.9.2, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: tensorflow-metadata 0.24.0 has requirement protobuf<4,>=3.7, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: tensorflow-hub 0.9.0 has requirement protobuf>=3.8.0, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: tensorflow-datasets 2.1.0 has requirement protobuf>=3.6.1, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: tensorboard 2.3.0 has requirement protobuf>=3.6.0, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: googleapis-common-protos 1.52.0 has requirement protobuf>=3.6.0, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement protobuf>=3.6.0, but you'll have protobuf 3.1.0 which is incompatible.
ERROR: google-api-core 1.16.0 has requirement protobuf>=3.4.0, but you'll have protobuf 3.1.0 which is incompatible.

In [2]:
import cirq

q = cirq.LineQubit(0)           # 實例化一個量子位元
simulator = cirq.Simulator()    # 實例化一個模擬器

X_circuit = cirq.Circuit(       # 建立一個包含量子NOT閘和測量的量子電路
    cirq.X(q),
    cirq.measure(q)
)
print(X_circuit)                # 在終端可視化輸出量子電路

# 使用模擬器對該量子電路進行20次的模擬測量
result = simulator.run(X_circuit, repetitions=20)
print(result)                   # 輸出模擬測量結果

H_circuit = cirq.Circuit(       # 建立一個包含阿達馬閘和測量的量子電路
    cirq.H(q),
    cirq.measure(q)
)
print(H_circuit)
result = simulator.run(H_circuit, repetitions=20)
print(result)

0: ───X───M───
0=11111111111111111111
0: ───H───M───
0=10010111100110010001


**混合量子 - 經典機器學習**

本節介紹混合量子 - 經典機器學習的基本概念，以及使用 TensorFlow Quantum 建立此類模型的方法。

In [5]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq

q = cirq.GridQubit(0, 0)
q_data = []
for i in range(100):
    x_i = cirq.Circuit(
        cirq.rx(np.random.rand() * np.pi)(q)
    )
    q_data.append(x_i)

NotFoundError: ignored

**參數化的量子電路（PQC）**

當我們在建立量子電路時使用了帶參數的量子閘，該參數可以自由調整，我們稱這樣的量子電路為參數化的量子電路。Cirq 支持結合 SymPy 這一 Python 下的符號運算函式庫實現參數化的量子電路，範例如下

In [ ]:
import sympy

theta = sympy.Symbol('theta')
q_model = cirq.Circuit(cirq.rx(theta)(q))

**範例：對量子資料集進行二分類**

在以下程式碼中，我們首先建立了一個量子資料集，其中一半的資料為基本態 \ket{0} 繞布洛赫球面的 x 軸逆時針旋轉 \frac{\pi}{2} 弧度（即 \frac{1}{\sqrt{2}} \ket{0} - \frac{i}{\sqrt{2}} \ket{1} ），另一半則為 \frac{3\pi}{2} 弧度（即 \frac{1}{\sqrt{2}} \ket{0} + \frac{i}{\sqrt{2}} \ket{1} ）。所有的資料均加入了繞 x,y 軸方向旋轉的，標準差為 \frac{\pi}{4} 的高斯噪聲。對於這個量子資料集，如果不加變換而直接測量，則所有資料都會和拋硬幣一樣等機率隨機坍縮到基本態 \ket{0} 和 \ket{1} ，從而無法區分。

In [ ]:
import cirq
import sympy
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq

q = cirq.GridQubit(0, 0)

# 準備量子資料集(q_data, label)
add_noise = lambda x: x + np.random.normal(0, 0.25 * np.pi)
q_data = tfq.convert_to_tensor(
    [cirq.Circuit(
        cirq.rx(add_noise(0.5 * np.pi))(q), 
        cirq.ry(add_noise(0))(q)
        ) for _ in range(100)] + 
    [cirq.Circuit(
        cirq.rx(add_noise(1.5 * np.pi))(q), 
        cirq.ry(add_noise(0))(q)
        ) for _ in range(100)]
)
label = np.array([0] * 100 + [1] * 100)

# 建立參數化的量子電路（PQC）
theta = sympy.Symbol('theta')
q_model = cirq.Circuit(cirq.rx(theta)(q))

# 建立量子層和經典全連接層
q_layer = tfq.layers.PQC(q_model, cirq.Z(q))
dense_layer = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)

# 使用Keras建立訓練流程。量子資料首先通過PQC，然後通過經典的全連接模型
q_data_input = tf.keras.Input(shape=() ,dtype=tf.dtypes.string)
expectation_output = q_layer(q_data_input)
classifier_output = dense_layer(expectation_output)
model = tf.keras.Model(inputs=q_data_input, outputs=classifier_output)

# 編譯模型，指定優化器、損失函數和評估指標，並進行訓練
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
model.fit(x=q_data, y=label, epochs=200)

# 輸出量子層參數（即theta）的訓練結果
print(q_layer.get_weights())